In [1]:
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Using Attention
Actually the data is not continuous so it should not use attention. But I dont care.

### Prepare the dataset

In [3]:
train_dataset = pd.read_csv('data/train_dataset.csv')
test_dataset = pd.read_csv('data/test_dataset.csv')

data_columns = ["V"+str(i) for i in range(1,29)]+["Amount"]
label_column ="Class"

X_train = train_dataset[data_columns]
X_test  = test_dataset[data_columns]

y_train = train_dataset[label_column]
y_test  = test_dataset[label_column]

x_train_tensor = torch.from_numpy(X_train.values).to(device)
y_train_tensor = torch.from_numpy(y_train.values).to(device)

x_test_tensor = torch.from_numpy(X_test.values).to(device)
y_test_tensor = torch.from_numpy(y_test.values).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

Train_dataset = DataLoader(Train_tensor, batch_size=512, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=512, shuffle=True)

### Simple attention block
This is the simple structure of attention block. There is not multi-heads or mask. If you want that, simply using torch.view() and torch.maskfilled

In [14]:
class AttentionBlock(nn.Module):
    def __init__(self, num_heads):
        super(AttentionBlock, self).__init__()

        self.num_heads = num_heads
        
        self.W_Q = nn.Linear(num_heads, num_heads)
        self.W_K = nn.Linear(num_heads, num_heads)
        self.W_V = nn.Linear(num_heads, num_heads)
        
    def forward(self, query, key, value):
        
        # softmax (QK^T/sqrt(d_k))V
        Q = self.W_Q(query)
        K = self.W_K(key)
        V = self.W_V(value)
        
        sm = torch.matmul(Q, K.transpose(-1,-2)) / (self.num_heads ** 0.5)
        
        out = torch.matmul(torch.softmax(sm, dim=-1),V)
        
        return out
        
        
        

### An encoder block in transformer architecture

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, num_heads, num_layers):
        super(EncoderBlock, self).__init__()
        
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.attention_blocks = AttentionBlock(num_heads)
        
        self.ffn = nn.Sequential(
            nn.Linear(num_heads, num_heads),
            nn.ReLU(),
            nn.Linear(num_heads, num_heads)
        )
        self.ln1 = nn.LayerNorm(num_heads)
        self.ln2 = nn.LayerNorm(num_heads)
        
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        
    def forward(self, x):
        
        temp = x
        x = self.attention_blocks(x,x,x)
        x += temp
        x = self.ln1(x)
        x = self.dropout1(x)
        
        temp = x
        
        x = self.ffn(x)
        x += temp
        x = self.ln2(x)
        x = self.dropout2(x)
        
        return x

### BERT-like architecture + Classification

In [18]:
class AttentionClassification(nn.Module):
    def __init__(self,input_dim, num_heads, num_layers):
        super(AttentionClassification, self).__init__()
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.input = nn.Linear(input_dim, num_heads)
        self.encoder_blocks = nn.ModuleList([EncoderBlock(num_heads, num_layers) for _ in range(num_layers)])
        self.fc_out = nn.Linear(num_heads, 1)
        
    def forward(self, x):
        x = self.inpu(x)
        # A super mini BERT
        for encoder in self.encoder_blocks:
            x = encoder(x)
        x = self.fc_out(x)
        return x

### Parameter

In [19]:
net = AttentionClassification(29,32, 4).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
epochs = 100

### Training

In [20]:
for epoch in range(epochs):
    for x, y in tqdm(Train_dataset):
        optimizer.zero_grad()
        output = net(x.float())

        loss = criterion(output, y.float().view(-1, 1))
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in tqdm(Test_dataset):
            output = net(x.float())
            predicted = (output >= 0.5).float()
            total += y.size(0)
            correct += (predicted == y.float().view(-1, 1)).sum().item()
        print(f"Epoch {epoch} Accuracy: {correct*100/total:.4f}%")

100%|██████████| 223/223 [00:02<00:00, 83.17it/s]


Epoch 0 Accuracy: 93.5969%


100%|██████████| 223/223 [00:02<00:00, 83.76it/s]


Epoch 1 Accuracy: 95.6492%


100%|██████████| 223/223 [00:02<00:00, 82.68it/s]


...

Epoch 58 Accuracy: 98.9747%


100%|██████████| 223/223 [00:02<00:00, 85.42it/s]


Epoch 59 Accuracy: 99.0407%


 57%|█████▋    | 508/889 [00:11<00:08, 43.64it/s]


KeyboardInterrupt: 